In [ ]:
!pip install --upgrade seaborn
!pip install --upgrade scipy
!pip install --upgrade numpy

#PROPER MATPLOTLIB INSTALLER
# !python -m pip uninstall matplotlib
!pip install matplotlib==3.1.3

# Note: The previous command,  
# !pip install --upgrade matplotlib 
# , !!!DOES NOT WORK ANYMORE!!! ... 
# This function of matplotlib has been deprecated for subsequent package updates

# MAKING IT PLOT/ Omissions of Google Colab Authorization Status...
# Don't Need file reader from abs_filepath, so ...
# !pip install --upgrade google-cloud


Requirement already up-to-date: seaborn in /usr/local/lib/python3.7/dist-packages (0.11.1)
     |████████████████████████████████| 27.4MB 149kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
     |████████████████████████████████| 15.3MB 337kB/s 
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.20.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5


     |████████████████████████████████| 13.1MB 344kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2


In [ ]:
import numpy as np, seaborn as sns, pandas as pd, scipy as sci, math, pylab
import matplotlib.transforms as mt, matplotlib.pyplot as plt
from matplotlib.transforms import offset_copy 

# Plots Class: EE517 Homework 7; Instructor: Dr. Jiming Song;    
# Date 'Due': 03-19-2021; Name: Kevin Spohn; Date Created: 03-15-21;
# Date Last Modified: 03-16-21

#Setup of Parameters:
try: 
    #Fundamental Constants:
    
    print("Debug 1\n")
    c = 3.0e8 
    mu0 = 4.0*np.pi*1.0e-7 
    # (i.) speed of light in free space (m/s) 
    # (ii.) Permeability of vacuum (in H/m)
    print("Debug 1a\n")
    epsilon0 = 8.854e-12  # (iii.) Permittivity of free space (in F/m)
    print("Debug 1b\n")
    eta = 376.7343092 # (iv.) Wave Impedance (Ohms)
    print("Debug 1c\n")
    freq = 3.0e8 # (v.) the wavelength should be about 1 meter 
    omega = 2*(np.pi)*freq 
    wavelength = (c/freq) # (vi., vii., viii.) Note:  cannot use 'lambda', due to 
    wave_k = 2*np.pi/wavelength # the lambda operator; which is applicable to 
    print("Debug 1d\n") # sets, dataframes and misc. datastreams...
    
    # Mesh Parameters: 
    # Note: all lengths are in unit of meter
    L_ant = 0.48*wavelength
    a = 2.0e-3*wavelength
    Ncell = 101 
    isrc = round(0.01+Ncell/2)          
    sourceType = 'd'  # location of source
    
    print("Debug 2\n")
    # Initialize matrices and geometry data
    if sourceType == 'f':
       Z_coax=50
       b= a*math.exp(2*np.pi*Z_coax/eta)
    print("Debug 3\n")

    A = np.zeros((Ncell,Ncell), dtype=float) 
    Iz = np.zeros((Ncell,1), dtype=float) 
    src = np.zeros((Ncell,1), dtype=float) 
    width = np.zeros((Ncell,1), dtype=float) 
    zcenter = np.zeros((Ncell,1), dtype=float) 
    dz = L_ant/Ncell 
    print("Debug 4\n")
    
    for n in range(1,Ncell,1):
        for m in range(1,Ncell, 1):
            if n == m:  
               # A[n, n] = 2*math.log(width[n]/a)-complex(0,1)*wave_k*width[n] 
               print("Debug 4a\n")
               ratio = width[n]/(2*a) 
               print("Debug 4b\n")
               A[n,n] = complex(2*math.log(ratio+sqrt(1+ratio**2)), 
               2*math.log(ratio+sqrt(1+ratio**2))-1*wave_k*width[n])
               print("Debug 4c\n") 
            else: 
               print("Debug 4d\n")
               R = sqrt(a**2 + (zcenter[m]-zcenter[n])**2) 
               print("Debug 4e\n")
               A[m,n] = complex(width[n]*np.cos(-1*wave_k*R)/R, 
               width[n]*np.sin(-1*wave_k*R)/R)
               print("Debug 4f\n")
        
            print("Debug 4g\n")
            zp = zcenter[n]+ 0.5*width[n] 
            zm = zcenter[n]- 0.5*width[n] 
            Rp = sqrt(a**2 + (zcenter[n]-zp)^2) 
            Rm = sqrt(a**2+ (zcenter[n]-zm)^2)
            print("Debug 4h\n") 
            qp = complex((zcenter[m]-zp)*(1)*np.cos(-1*wave_k*Rp)/(Rp**3), 
                        (zcenter[m]-zp)*(wave_k*Rp)*np.sin(-1*wave_k*Rp)/(Rp**3))
            qm = complex((zcenter[m]-zp)*(1)*np.cos(-1*wave_k*Rm)/(Rm**3), 
                        (zcenter[m]-zp)*(wave_k*Rm)*np.sin(-1*wave_k*Rm)/(Rm**3))
            print("Debug 4i\n")
            A[m, n] = complex(wave_k^2*A[m,n].real + (qp-qm).real,   
                              wave_k^2*A[m,n].imag + (qp-qm).imag)  
            print("Debug 4j\n")

    print("Debug 5\n")
    A = np.linalg.inv(A)
    # filling RHS

    print("Debug 6\n")
    coeff = complex(0, 4*np.pi*(-1)*omega*epsilon0)   
    if source == 'd':
       src[isrc] = coeff/width[isrc]   
    else:
       for m in range(1, Ncell,1):
           Ra = sqrt(a**2 + (zcenter[m]-zcenter[isrc])**2)
           Rb = sqrt(b**2 + (zcenter[m]-zcenter[isrc])**2 )
           src[m] = complex(coeff.real/(2*math.log(b/a))*(
           (np.cos(-1*wave_k*Ra)/Ra)-(np.cos(-1*wave_k*Rb)/Rb) ), 
           coeff.imag/(2*math.log(b/a))*(
           (np.sin(-1*wave_k*Ra)/Ra)-(np.sin(-1*wave_k*Rb)/Rb) ) )  
          
    print("Debug 7\n")
    
    # Solving the matrix equations
    Iz = np.matmul(A, src) # output input impedance Matrix
    Zin = 1/Iz[isrc] #specific isrc as index, assuming nonzero... whatever
    
    #Problem 1b.  
    print("Input impedance: (%8.3f %8.3f)ohms\n" % (Zin.real, Zin.imag))  
    # plot the current distribution, and Figure 1.
    print("Debug 8\n")

    # plot(zcenter*2/leng_ant, 1000*abs(Iz), 'r-','MarkerSize',12,'linewidth',2)  
    # set (gca, 'fontsize', 16)  
    # legend('MOM result')  
    # title('Wire Ant, where L_ant =0.48*wavelength(m), a = 2.0e-3*wavelength(m), and f=3.0e8Hz')  
    # xlabel('z/(L/2)')  
    # ylabel('Current (mA)')  

    # calculate far field
    print("Debug 9\n")
    r = 1.0e7*wavelength  
    theta_deg = np.linspace(start=-1*np.pi, stop=np.pi, num=200)  
    ctemp = complex(0)
    
    (complex(0,1)*omega*mu0/(4*np.pi))*(math.exp(-1*complex(0,1)*wave_k*r)/r)  

    for it in range(1,length(theta_deg), 1):
        phase = math.exp(complex(0,1)*wave_k*zcenter*np.cos(theta_deg[it]))  
        ve = phase*np.matmul(Iz, width)  
        E[it] = sin(theta_deg[it])*ctemp*np.sum(ve)  
    
    # Radiation intensity
    ri = (0.5/eta)*(r**2)*(np.matmul(E, np.linalg.inv(E)))  
    power_far = ri/max(ri)  
    power_far_dB = 10*math.log10(power_far+1e-40)  

    # power pattern in xy-plot
    #Figure 2: 

    # figure  
    # plot(theta*180/pi, power_far, 'r-','MarkerSize',12,'linewidth',2)  
    # set (gca, 'fontsize', 16)  
    # legend('MOM result')  
    # title('Wire Ant, where L_ant = 0.48*wavelength(m), a = 2.0e-3*wavelength(m), and f=3.0e8Hz')  
    # ylabel('Power Pattern')  
    # xlabel('theta (degrees)')  
    # axis([-360 360 0.0 20])  
    
    # power pattern in x-y coordinate
    
    # figure;
    # plot(theta*180/pi, power_far, 'r-','MarkerSize',12,'linewidth',2);
    # set (gca, 'fontsize', 16);
    # legend('MOM result');
    # title('Wire Ant, where L_ant =0.48*wavelength(m), a = 2.0e-3*wavelength(m), and f=3.0e8Hz')  
    # ylabel('Power Pattern');
    # xlabel('theta (degrees)');
    # axis([-180 180 0.0 1.2]);

    # power pattern in polar coordinate
    
    # figure;
    # HandleCurve=polar(theta,power_far, 'r-');
    # set(HandleCurve,'LineWidth',2);
    # set (gca, 'fontsize', 16);
    # legend('MOM result');
    # title('Wire Ant, where L_ant=0.48*wavelength(m), a = 2.0e-3*wavelength(m), and f=3.0e8Hz')  
    # ylabel('Power Pattern');
    
    # power pattern in polar coordinate with a 90 degree turn

    # figure;
    # HandleCurve = polar(theta,power_far, 'r-');
    # CurrentAxesHandle = gca;
    # set(CurrentAxesHandle, 'View', [90,-90]);
    # set(HandleCurve,'LineWidth',2);
    # set (gca, 'fontsize', 16);
    # legend('MOM result');
    # title('Wire Ant, where L_ant =0.48*wavelength(m), a = 2.0e-3*wavelength(m), and f=3.0e8Hz')  
    # ylabel('Power Pattern');
    
    # % power pattern in polar coordinate in log scale
    # figure;
    # polarhg(theta,power_far_dB,'tdir','clockwise','rlim',[-40 0],...
    #   'rtick',[-40 -30 -20 -10 -3 0],'tstep',30,'torig','up','color',...
    #   'r','linestyle','-');
    # set (gca, 'fontsize', 16);
    # title('Power Pattern in dB');
     
    # figure;
    # polarhg(theta,power_far,'tdir','clockwise','rlim',[0 1],...
    #    'rtick',[0.2 0.4 0.6 0.8 1.0],'tstep',30,'torig','up','color',...
    #    'r','linestyle','-');
    # set (gca, 'fontsize', 16);
    # title('Power Pattern');

except Exception: 
   print("Exception, error has occurred. Check the stack trace and try again...\n")

Debug 1

Debug 1a

Debug 1b

Debug 1c

Debug 1d

Debug 2

Debug 3

Debug 4

Debug 4a

Debug 4b

Exception, error has occurred. Check the stack trace and try again...



In [ ]:
# fig1 = plt.figure(figsize=(6,6))
#     axF1 = fig1.add_subplot(111,polar=True)
#     axF1.plot(angleRadData, S21_dB, label='S21(dB)') #theta,power_gain(dB)
#     axF1.set_ylim(-90,-40)
#     axF1.set_yticks(np.arange(-90,-40,10)) 
#     plt.title("Lab 4 Figure 1. Polar Plot of S21(dB)")
#     plt.legend() 
#     plt.savefig("Lab4_Figure1_Python.png")
#     plt.show()
#     print("Fig 1 printed...\n")